# Mismatch detection
>Antonio Colás Nieto, @acolasn, anconi.1999@gmail.com

We will run standard synchronization code and attempt to find and analyse mismatch responses.  

Taken from `demo_analysis.ipynb`, and hoping that I need to change as little as possible. 

In [ ]:
%reload_ext autoreload
%autoreload 2

import functools

print = functools.partial(print, flush=True)

import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import pickle
from cottage_analysis.analysis import (
    mismatch)
import flexiznam as flz


In [ ]:
# Example session
project = "663214d08993fd0b6e6b5f1d"
mouse = "BRAC9057.4j"
session = "BRAC9057.4j_S20240516"
#RECORDING = "R171146_KellerTube"
protocol = "KellerTube"
MESSAGES = "harpmessage.bin"
flexilims_session = flz.get_flexilims_session(project_id=project)
# all_protocol_recording_entries = generate_filepaths.get_all_recording_entries(project=project,
#                                                                               mouse=mouse,
#                                                                               session=session,
#                                                                               protocol=protocol,
#                                                                               flexilims_session=flexilims_session)

# # DO NOT RUN THIS FUNCTION (TAKES 2hrs ish): to find monitor frames from photodiode signal
# find_monitor_frames(project=project,
#                     mouse=mouse,
#                     session=session,
#                     protocol=protocol,
#                     all_protocol_recording_entries=None,
#                     irecording=None,
#                     flexilims_session=None)

## Calculating  $dF/F$ only for one recording  
  
We first split F and Fneu for all recordings, and then

## Synchronization

Ideally, this would work out of the bat. 

In [ ]:
# Generate synchronisation dataframes

vs_df_all, imaging_df_all, recordings = mismatch.sync_all_recordings(
    session_name=session,
    flexilims_session=flexilims_session,
    project=project,
    filter_datasets={"anatomical_only": 3},
    recording_type="two_photon",
    protocol_base="KellerTube",
    photodiode_protocol=5,
    return_volumes=True,
)

# Analysis: choose a recording from the session

In [ ]:
closed_loop = imaging_df_all[2]

In [ ]:
#plt.plot(imaging_df_all["mouse_z"]/imaging_df_all["mismatch_mouse_z"])
plt.plot(closed_loop["mouse_z"])

Good. 

## Finding mismatches

In [ ]:
closed_loop["mousez_dif"]  = np.zeros(len(closed_loop["mouse_z"]))
closed_loop.loc[1:, 'mousez_dif'] = np.diff(closed_loop['mouse_z'])


In [ ]:
closed_loop["mismz_dif"]  = np.zeros(len(closed_loop["mouse_z"]))
closed_loop.loc[1:, 'mismz_dif'] = np.diff(closed_loop['mismatch_mouse_z'])

In [ ]:
closed_loop["mism_ratio"] = closed_loop["mousez_dif"]/closed_loop["mismz_dif"]

In [ ]:
plt.plot(closed_loop["mism_ratio"][720:740])

In [ ]:
closed_loop["mismatch"] = ((closed_loop["mism_ratio"] > 1.2) | (closed_loop["mism_ratio"] < -1000)).astype(int)

#To catch the fact that it's -inf sometimes during a mismatch

In [ ]:
up= 0
down =  -1

plt.plot(closed_loop["mism_ratio"][up:down])
plt.plot(closed_loop["mismatch"][up:down])

closed_loop["mism_ratio"][up:down]

In [ ]:
closed_loop.drop(columns = {"mismz_dif", "mousez_dif"})

## Defining mismatch window for raster

By design, 5 frames before and 10 after the mismatch onset frame. The length is 15 frames. 

In [ ]:
# Create a new column initialized to 0
closed_loop['range_indicator'] = 0

#Make a diff to look at starting frames
closed_loop["start_mismatch"]=np.zeros(len(closed_loop["mismatch"]))

closed_loop.loc[1:,"start_mismatch"] = np.diff(closed_loop["mismatch"])

# Find indices where 'indicator' is 1
indices = closed_loop.index[closed_loop['start_mismatch'] == 1].tolist()
print(indices)

# Set range_indicator to 1 for 5 rows before and after each index where 'indicator' is 1
for idx in indices:
    start = max(idx - 5, 0)
    end = min(idx + 10, len(closed_loop['mismatch']) - 1)
    print((start,  end))
    closed_loop.loc[start:end, 'range_indicator'] = 1

For every neuron, we want to find a point in time where the mismatch begins and calculate the average of its responses to a mismatch

In [ ]:
# Create the initial DataFrame with range_indicator
neurons_df = pd.DataFrame({"range_indicator": closed_loop["range_indicator"].copy()})

# Extract the number of neurons
neurons = closed_loop["dffs"][0].shape[1]

# Create a DataFrame for the neurons data
neuron_data = pd.DataFrame(
    {f"neuron{neuron}": [closed_loop["dffs"][i][0][neuron] for i in range(len(closed_loop["mismatch"]))]
     for neuron in range(neurons)}
)

# Concatenate the range_indicator and neuron data
neurons_df = pd.concat([neurons_df, neuron_data], axis=1)

In [ ]:
mismatches_per_neuron = list(np.zeros(neurons))

neurons_df["start_mismatch"]=np.zeros(len(neurons_df["range_indicator"]))

neurons_df.loc[1:,"start_mismatch"] = np.diff(neurons_df["range_indicator"])

n_mismatches  =  len(neurons_df["start_mismatch"][neurons_df["start_mismatch"]==1])

print(n_mismatches)

for i in range(neurons):
    mismatches_per_neuron[i] = np.zeros((n_mismatches, 15))



In [ ]:


# Initialize variables to track the start and end of intervals
in_interval = False
start_idx = None

# Iterate through the DataFrame to identify intervals
idx_mismatch = -1
for idx, row in neurons_df.iterrows():
    if row['range_indicator'] == 1 and not in_interval:
        # Start of a new interval
        start_idx = idx
        in_interval = True
        idx_mismatch += 1
        print(f"This is mismatch {idx_mismatch}")
    elif row['range_indicator'] == 0 and in_interval:
        # End of the current interval
        end_idx = idx-1
        for neuron in range(neurons):
            mismatches_per_neuron[neuron][idx_mismatch, :] = neurons_df[f"neuron{neuron}"][start_idx:end_idx]
        in_interval = False
        print(f"start and end idx: {(start_idx, end_idx)}")

In [ ]:
mismatch_raster = np.zeros((neurons, 15))

for i in range(neurons):
    mismatch_raster[i, :] = np.mean(mismatches_per_neuron[i], axis = 0)

#print(mismatch_raster.shape)

# Define a function to calculate the difference for each row
def calculate_difference(row):
    first_5_sum = np.sum(row[0:5])
    last_5_sum = np.sum(row[6:11])
    return last_5_sum-first_5_sum

# Calculate differences for each row
differences = np.apply_along_axis(calculate_difference, 1, mismatch_raster)
#print(differences[0:10])

# Get the sorted indices based on the differences (larger differences first)
sorted_indices = np.argsort(-differences)
#print(sorted_indices[0:10])

# Sort the array based on the calculated differences
sorted_mismatch_raster = mismatch_raster[sorted_indices]

start = 0
end = 100

fig = plt.figure(figsize=(30,10),facecolor='w') 
ax = fig.add_subplot(111)
im = ax.imshow(sorted_mismatch_raster[0:100])



ax.set_title(f"Raster plot of first {end} neurons aligned to mismatch")
ax.set_xlabel("Frames")
ax.set_ylabel("Neurons")
fig.colorbar(im, label  =  "dff")
ax.axvline(5, color = "grey")

In [ ]:
plt.plot(differences[sorted_indices])


In [ ]:
pop_response  = np.mean(sorted_mismatch_raster[0:100], axis = 0)
plt.plot(pop_response)

# Now sorting a la Keller

Evaluate significant mismatch modulation  by producing a series of  random events and looking at the differential modullation from mismatches. 

In [ ]:
def generate_random_events(n_frames, n_events = 100):
    return [random.randint(0, n_frames-1) for _ in range(n_events)]

In [ ]:
n_frames = len(closed_loop)
events = generate_random_events(n_frames)

In [ ]:
closed_loop["randevents"] = 0
closed_loop.loc[events, "randevents"] = 1
plt.plot(closed_loop["randevents"])

In [ ]:
rand_rec, indices = mismatch.create_mismatch_window(closed_loop, window_start = 0, window_end = 5, event  = "randevents")

In [ ]:
neurons, rand_neurodf = mismatch.build_neurons_df(rand_rec)

In [ ]:
rand_misperneuron = mismatch.build_mismatches_per_neuron_list(neurons, rand_neurodf, window_start = 0, window_end = 5, indices = indices)

In [ ]:
rand_raster = mismatch.raster(neurons, rand_misperneuron, window_start = 0, window_end= 5)

In [ ]:
rand_avg = np.mean(rand_raster, axis = 1)
mismatch_avg = np.mean(mismatch_raster[:, 8:13], axis = 1)
modulation_raster = mismatch_avg-rand_avg

In [ ]:
sorted_indices = np.argsort(-modulation_raster)
#print(sorted_indices[0:10])

plt.plot(modulation_raster[sorted_indices])

In [ ]:


# Sort the array based on the calculated differences
sorted_mismatch_raster = mismatch_raster[sorted_indices]

# Define the range you want to cap
vmin = -0.5
vmax = 1

start = 0
end = 100
fig = plt.figure(figsize=(30, 10), facecolor='w')
ax = fig.add_subplot(111)
im = ax.imshow(sorted_mismatch_raster, cmap='coolwarm', vmin=vmin, vmax=vmax, aspect =  "auto")


ax.set_title(f"Raster plot of neurons aligned to mismatch")
ax.set_xlabel("Frames")
ax.set_ylabel("Neurons")
fig.colorbar(im, label="dff")
ax.axvline(5, color="grey")
plt.show()

In [ ]:
pop_response  = np.mean(sorted_mismatch_raster, axis = 0)
plt.plot(pop_response)

In [ ]:
closed_loop, indices = mismatch.create_mismatch_window(closed_loop, window_start = 5, window_end = 20)
neurons, neurons_df = mismatch.build_neurons_df(closed_loop)
misperneuron = mismatch.build_mismatches_per_neuron_list(neurons, neurons_df, window_start = 5, window_end = 20)
mismatch_raster = mismatch.raster(neurons, misperneuron, window_start = 5, window_end = 20)
rand_raster = mismatch.make_rand_raster(closed_loop, n_events = 200, window_end = 10)
sorted_mismatch_raster = mismatch.modulation_sort_raster(rand_raster, mismatch_raster)
mismatch.plot_raster(sorted_mismatch_raster, vmin = -0.5,  vmax = 0.8)

In [ ]:
mismatch.plot_raster(sorted_mismatch_raster, vmin = -0.5,  vmax = 0.8)